In [1]:
from scipy.io import loadmat
data = loadmat('/home/hnaxiong/ser/exe4/dataset3.mat')
keys = data.keys()
keys

dict_keys(['__header__', '__version__', '__globals__', 'theta_vk_i', 'r_i_vk_i', 't', 'w_vk_vk_i', 'v_vk_vk_i', 'rho_i_pj_i', 'y_k_j', 'C_c_v', 'rho_v_c_v', 'w_var', 'v_var', 'y_var', 'fu', 'fv', 'cu', 'cv', 'b'])

In [20]:
fu = data['fu'].item()
fv = data['fv'].item()
cu = data['cu'].item()
cv = data['cv'].item()
b = data['b'].item()
Pj_i_true = data['rho_i_pj_i']


In [37]:
import numpy as np
#define stereo camera projection matrix
M = np.array([[fu, 0, cu, 0],
     [0, fv, cv, 0],
     [fu, 0, cu, -fu*b],
     [0, fv, cu, 0]])
# convert 3D point into 2d stereo point pair
def Cam_Projection(pt):
    x = pt[0]
    y = pt[1]
    z = pt[2]
    pt = np.array([[x],[y],[z],[1]])
    pt_2d = np.dot(M / z , pt)
    return pt_2d

# convert ground truth point into stereo point pairs
Pj_in_cam = []
for i in range(20):
    pt = [Pj_i_true[0,i],Pj_i_true[1,i],Pj_i_true[2,i]]
    pt_2d = Cam_Projection(pt)
    Pj_in_cam.append(pt_2d)


In [ ]:
import cv2
import numpy as np

# 生成示例数据
num_3d_points = 20
num_frames = 100
points_3d = np.random.rand(num_3d_points, 3)  # 20个静止的3D点
points_2d = np.random.rand(num_frames, num_3d_points, 2)  # 100个时刻下的2D点

# 设置视频参数
frame_width = 800
frame_height = 600
fps = 30

# 创建视频写入器
video_writer = cv2.VideoWriter('point_motion.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# 遍历每一帧
for frame_index in range(num_frames):
    frame = np.ones((frame_height, frame_width, 3), np.uint8) * 255  # 创建白色背景
    
    # 将3D点投影到当前帧的2D坐标
    for point_index in range(num_3d_points):
        u = int(points_2d[frame_index, point_index, 0] * frame_width)
        v = int(points_2d[frame_index, point_index, 1] * frame_height)
        cv2.circle(frame, (u, v), 5, (0, 0, 255), -1)  # 在当前帧上绘制红色的2D点
        
        # 在当前帧上绘制3D点（以黄色十字标记）
        x = int(points_3d[point_index, 0] * frame_width)
        y = int(points_3d[point_index, 1] * frame_height)
        cv2.drawMarker(frame, (x, y), (0, 255, 255), markerType=cv2.MARKER_CROSS, markerSize=10)

    # 将当前帧写入视频
    video_writer.write(frame)

# 释放视频写入器
video_writer.release()

print("视频已保存为 'point_motion.mp4'")
